# A/B Testing the Udacity Website

In these exercises, we'll be analyzing data on user behavior from an experiment run by Udacity, the online education company. More specifically, we'll be looking at a test Udacity ran to improve the onboarding process on their site.

Udacity's test is an example of an "A/B" test, in which some portion of users visiting a website (or using an app) are randomly selected to see a new version of the site. An analyst can then compare the behavior of users who see a new website design to users seeing their normal website to estimate the effect of rolling out the proposed changes to all users. While this kind of experiment has it's own name in industry (A/B testing), to be clear it's just a randomized experiment, and so everything we've learned about potential outcomes and randomized experiments apply here. 

(Udacity has generously provides the data from this test under an Apache open-source license, and you can find their [original writeup here](https://www.kaggle.com/tammyrotem/ab-tests-with-python/notebook). If you're interested in learning more on A/B testing in particular, it seems only fair while we use their data to flag they have a full course on the subject [here](https://www.udacity.com/course/ab-testing--ud257).)

## Udacity's Test

The test [is described by Udacity as follows](https://www.kaggle.com/tammyrotem/ab-tests-with-python/notebook): 

At the time of this experiment, Udacity courses currently have two options on the course overview page: "start free trial", and "access course materials".

**Current Conditions Before Change**

- If the student clicks "start free trial", they will be asked to enter their credit card information, and then they will be enrolled in a free trial for the paid version of the course. After 14 days, they will automatically be charged unless they cancel first.
- If the student clicks "access course materials", they will be able to view the videos and take the quizzes for free, but they will not receive coaching support or a verified certificate, and they will not submit their final project for feedback.

**Description of Experimented Change**

- In the experiment, Udacity tested a change where if the student clicked "start free trial", they were asked how much time they had available to devote to the course.
- If the student indicated 5 or more hours per week, they would be taken through the checkout process as usual. If they indicated fewer than 5 hours per week, a message would appear indicating that Udacity courses usually require a greater time commitment for successful completion, and suggesting that the student might like to access the course materials for free.
- At this point, the student would have the option to continue enrolling in the free trial, or access the course materials for free instead. This [screenshot](images/udacity_checkyoureready.png) shows what the experiment looks like.

**Udacity's Hope is that...**:

> this might set clearer expectations for students upfront, thus reducing the number of frustrated students who left the free trial because they didn't have enough time -- without significantly reducing the number of students to continue past the free trial and eventually complete the course. If this hypothesis held true, Udacity could improve the overall student experience and improve coaches' capacity to support students who are likely to complete the course.



## Gradescope Autograding

Please follow [all standard guidance](https://www.practicaldatascience.org/ids720_specific/autograder_guidelines.html) for submitting this assignment to the Gradescope autograder, including storing your solutions in a dictionary called `results` and ensuring your notebook runs from the start to completion without any errors.

For this assignment, please name your file `exercise_abtesting.ipynb` before uploading.

You can check that you have answers for all questions in your `results` dictionary with this code:

```python
assert set(results.keys()) == {
    "ex4_avg_oec",
    "ex5_avg_guardrail",
    "ex7_ttest_pvalue",
    "ex9_ttest_pvalue_clicks",
    "ex10_num_obs",
    "ex11_guard_ate",
    "ex11_guard_pvalue",
    "ex11_oec_ate",
    "ex11_oec_pvalue",
    "ex14_se_treatment",
}
```


### Submission Limits

Please remember that you are **only allowed THREE submissions to the autograder.** Your last submission (if you submit 3 or fewer times), or your third submission (if you submit more than 3 times) will determine your grade Submissions that error out will **not** count against this total.

## Import the Data

### Exercise 1

Begin by importing Udacity's data on user behavior [here.](https://github.com/nickeubank/MIDS_Data/tree/master/udacity_AB_testing) 

There are TWO datasets for this test — one for the control data (users who saw the original design), and one for treatment data (users who saw the experimental design). Udacity decided to show their test site to 1/2 of visitors, so there are roughly the same number of users appearing in each dataset (though this is not a requirement of AB tests).

Please remember to load the data directly from github to assist the autograder.

In [ ]:
import pandas as pd

results = {}
df_ctrl = pd.read_csv(
    "https://github.com/nickeubank/MIDS_Data/raw/refs/heads/master/udacity_AB_testing/control_data.csv"
)
df_exp = pd.read_csv(
    "https://github.com/nickeubank/MIDS_Data/raw/refs/heads/master/udacity_AB_testing/experiment_data.csv"
)

### Exercise 2

Explore the data. Can you identify the unit of observation of the data (e.g. what is represented by each row)?

To be clear, the columns represent stages in a user funnel:

- Some number of users arrive at the website and are counted as Pageviews,
- Some portion of those users then click to enroll (and are counted as clicks),
- Some portion of those users then actually enroll in the free trial (after seeing an informational popup, in the case of treatment individuals),
- Finally some portion of those users end up paying at the end of the free trial period.

(Note this is not the only way that A/B test data can be collected and/or reported — this is just what Udacity provided, presumably to help address privacy concerns.)

In [2]:
df_ctrl.head(5)

,Date,Pageviews,Clicks,Enrollments,Payments
0,"Sat, Oct 11",7723,687,134.0,70.0
1,"Sun, Oct 12",9102,779,147.0,70.0
2,"Mon, Oct 13",10511,909,167.0,95.0
3,"Tue, Oct 14",9871,836,156.0,105.0
4,"Wed, Oct 15",10014,837,163.0,64.0


In [3]:
df_exp.head(5)

,Date,Pageviews,Clicks,Enrollments,Payments
0,"Sat, Oct 11",7716,686,105.0,34.0
1,"Sun, Oct 12",9288,785,116.0,91.0
2,"Mon, Oct 13",10480,884,145.0,79.0
3,"Tue, Oct 14",9867,827,138.0,92.0
4,"Wed, Oct 15",9793,832,140.0,94.0


In [4]:
print(df_exp.shape)
print(df_ctrl.shape)

(37, 5)
(37, 5)


> Each row represents a day - and the columns represent the total count at each stage of the funnel (for the day respectively).

## Pick your measures

### Exercise 3

The easiest way to analyze this data is to stack it into a single dataset where each observation is a day-treatment-arm (so you should end up with two rows per day, one for those who are in the treated groups, and one for those who were in the control group). Note that currently nothing in the data identifies whether a given observation is a treatment group observation or a control group observation, so you'll want to make sure to add a "treatment" indicator variable.

The variables in the data are:

- Pageviews: number of unique users visiting homepage
- Clicks: number of those users clicking "Start Free Trial"
- Enrollments: Number of people enrolling in trial
- Payments: Number of people who eventually pay for the service. Note the `payment` column reports payments for the users who first visited the site on the reported date, not payments occurring on the reported date.

In [ ]:
df_ctrl["treated"] = 0
df_exp["treated"] = 1
df_both = pd.concat([df_ctrl, df_exp], axis=0, ignore_index=True)

In [6]:
df_ctrl.head(5)

,Date,Pageviews,Clicks,Enrollments,Payments,treated
0,"Sat, Oct 11",7723,687,134.0,70.0,0
1,"Sun, Oct 12",9102,779,147.0,70.0,0
2,"Mon, Oct 13",10511,909,167.0,95.0,0
3,"Tue, Oct 14",9871,836,156.0,105.0,0
4,"Wed, Oct 15",10014,837,163.0,64.0,0


In [7]:
df_both.head(5)
print(df_both.shape)

(74, 6)


### Exercise 4

Given Udacity's goals, what outcome are they hoping will be impacted by their manipulation?

Or, to ask the same question in the language of the Potential Outcomes Framework, what is their $Y$?

Or to ask the same question in the language of Kohavi, Tang and Xu, what is their *Overall Evaluation Criterion (OEC)*?

(I'm only asking one question, I'm just trying to phrase it using different terminologies we've encountered to help you see how they all fit together)

When you feel like you have your answer, please compute it. Store the average value of the variable in `results` under the key `ex4_avg_oec`. **Please round your answer to 4 decimal places.**

NOTE: You'll probably notice you have two choices to make when it comes to actually computing the OEC. 

- You could probably imagine either computing a ratio or a difference of two things — please calculate the difference.
- You may also be unsure whether to normalize by `Clicks`. Normalizing by clicks will help account for variation that comes from day-to-day variation in users, so it's a good thing to do. With infinite data, you'd expect to get the same results without normalizing by `Clicks` (since on average the same share of users are in each arm of the experiment), but for finite data it's a good strategy. Note that this is only ok because users make the choice to click or not *before* they see different versions of the website (it is "pre-treatment").

Just to make sure you're on track, your measure should have an average value of *about* 9%.

In [ ]:
df_both["diff"] = df_both["Payments"] - df_both["Enrollments"]
df_both["click_diff_ratio"] = df_both["diff"] / df_both["Clicks"]
oec_ctrl = df_both.loc[df_both["treated"] == 0, "click_diff_ratio"].mean()
oec_treat = df_both.loc[df_both["treated"] == 1, "click_diff_ratio"].mean()

print("OEC (control):", oec_ctrl)
print("OEC (treatment):", oec_treat)

results["ex4_avg_oec"] = round(float(oec_ctrl), 4)

OEC (control): -0.10208153306160457
OEC (treatment): -0.08619380802214803


In [ ]:
print(df_both["click_diff_ratio"].mean())

-0.0941376705418763


The outcome they hope to impact is the extent to which users who click “Start Free Trial” ultimately continue through the trial and convert into paying customers. In other words, Udacity is trying to influence the drop-off between enrollment in the free trial and eventual payment, with the aim of setting clearer expectations upfront so that fewer users enroll and then leave the trial due to insufficient time, while preservin meaningful conversions to paid users.

### Exercise 5

Given Udacity's goals, what outcome are they hoping will *not* be impacted by their manipulation? In other words, what do they want to measure to ensure their treatment doesn't have unintended negative consequences that might be really costly to their operation?

Note that while this isn't how Kohavi, Tang, and Xu use the term "guardrail metrics" — they usually use the term to refer to things we measure to ensure the experiment is working the way it should — some people would also use the term "guardrail metrics" for something that could be impacted even if the experiment is working correctly, but which the organization wants to track to ensure they aren't impacted because they are deemed really important.

Again, please normalize by `Clicks`. Store the average value of this guardrail metric as `ex5_avg_guardrail` and **round your answer to 4 decimal places.**

In [11]:
df_both["click_pay_ratio"] = df_both["Payments"] / df_both["Clicks"]

guard_ctrl = df_both.loc[df_both["treated"] == 0, "click_pay_ratio"].mean()
guard_treat = df_both.loc[df_both["treated"] == 1, "click_pay_ratio"].mean()

print("Guardrail (control):", guard_ctrl)
print("Guardrail (treatment):", guard_treat)

results["ex5_avg_guardrail"] = round(float(guard_ctrl), 4)

Guardrail (control): 0.11826943663824471
Guardrail (treatment): 0.11337257964843535


Udacity’s guardrail metric checks that the treatment does not reduce monetization among users who click，and we use Payments / Clicks to measure the click-to-payment conversion rate. Also, in the control group it averages about 0.1183, meaning roughly 11.83% of clickers eventually become paying customers. Udacity tracks this as a guardrail because a meaningful drop would be an unintended and costly negative consequence.

## Validating The Data

### Exercise 6

Whenever you are working with experimental data, the first thing you want to do is verify that users actually were randomly sorted into the two arms of the experiment. In this data, half of users were supposed to be shown the old version of the site and half were supposed to see the new version.

`Pageviews` tells you how many unique users visited the welcome site we are experimenting on. `Pageviews` is what is sometimes called an "invariant" or "guardrail" variable, meaning that it shouldn't vary across treatment arms—after all, people have to visit the site before they get a chance to see the treatment, so there's no way that being assigned to treatment or control should affect the number of pageviews assigned to each group.

"Invariant" variables are also an example of what are known as a "pre-treatment" variable, because pageviews are determined before users are manipulated in any way. That makes it analogous to gender or age in experiments where you have demographic data—a person's age and gender are determined before they experience any manipulations, so the value of any pre-treatment attributes should be the same across the two arms of our experiment. This is what we've previously called "checking for balance," If pre-treatment attributes aren't balanced, then we may worry our attempt to randomly assign people to different groups failed.  Kohavi, Tang and Xu call this a "trust-based guardrail metric" because it helps us determine if we should trust our data.

To test the quality of the randomization, calculate the average number of pageviews for the treated group and for the control group. Do they look similar?


In [12]:
from scipy import stats

pv_ctrl = df_both.loc[df_both["treated"] == 0, "Pageviews"].mean()
pv_treat = df_both.loc[df_both["treated"] == 1, "Pageviews"].mean()

print("Avg Pageviews (control):", pv_ctrl)
print("Avg Pageviews (treatment):", pv_treat)

Avg Pageviews (control): 9339.0
Avg Pageviews (treatment): 9315.135135135135


The average number of pageviews is very similar between the control group (about 9339) and the treatment group (about 9315), suggesting that users were evenly and randomly assigned to the two arms of the experiment and that the randomization appears to be valid.

### Exercise 7

"Similar" is a tricky concept -- obviously, we expect *some* differences across groups since users were *randomly* divided across treatment arms. The question is whether the differences between groups are larger than we'd expect to emerge given our random assignment process. To evaluate this, let's use a `ttest` to test the statistical significance of the differences we see. 

**Note**: Remember that scipy functions don't accept `pandas` objects, so you use a scipy function, you have to pass the numpy vectors underlying your data with the `.values` operator (e.g. `df.my_column.values`). 

Does the difference in `pageviews` look statistically significant?

Store the resulting p-value in `ex7_ttest_pvalue` **rounded to four decimal places.**

In [13]:
pv_ctrl_vec = df_both.loc[df_both["treated"] == 0, "Pageviews"].values
pv_treat_vec = df_both.loc[df_both["treated"] == 1, "Pageviews"].values

pval = stats.ttest_ind(pv_treat_vec, pv_ctrl_vec, equal_var=False).pvalue
print("T-test p-value (Pageviews):", pval)

results["ex7_ttest_pvalue"] = round(float(pval), 4)

T-test p-value (Pageviews): 0.8877041856788624


With a p-value of 0.8877, the difference in pageviews between the treatment and control groups is not statistically significant, which suggests the two groups received comparable traffic and supports the assumption that random assignment worked as intended.

### Exercise 8

`Pageviews` is not the only "pre-treatment" variable in this data we can use to evaluate balance/use as a guardrail metric. What other measure is pre-treatment? Review the description of the experiment if you're not sure.

Another pre-treatment measure is Clicks. Users decide whether or not to click “Start Free Trial” before they see the experimental prompt, so clicks are determined prior to any treatment and should be balanced across the treatment and control groups if randomization worked correctly.

### Exercise 9

Check if the other pre-treatment variable is also balanced. Store the p-value of your test of difference in `results` under the key `"ex9_ttest_pvalue_clicks"` **rounded to four decimal places.**

In [14]:
click_ctrl_vec = df_both.loc[df_both["treated"] == 0, "Clicks"].values
click_treat_vec = df_both.loc[df_both["treated"] == 1, "Clicks"].values
pval_clicks = stats.ttest_ind(click_treat_vec, click_ctrl_vec, equal_var=False).pvalue
print("T-test p-value (Clicks):", pval_clicks)
results["ex9_ttest_pvalue_clicks"] = round(float(pval_clicks), 4)

T-test p-value (Clicks): 0.9263950638615311


The p-value for the difference in clicks is 0.9264, indicating no statistically significant difference between the treatment and control groups, which suggests clicks are well balanced.

## Estimating the Effect of Experiment

### Exercise 10

Now that we've validated our randomization, our next task is to estimate our treatment effect. First, though, there's an issue with your data you've been able to largely ignore until now, but which you should get a grip on before estimating your treatment effect — can you tell what it is and what you should do about it?

Store the number of observations in your data *after* you've addressed this in `ex10_num_obs` (this is mostly meant as a way to sanity check your answer with autograder).

In [ ]:
df_clean = df_both.dropna(subset=["Payments", "Enrollments"]).copy()


print("Before cleaning:", df_both.shape)
print("After cleaning :", df_clean.shape)

results["ex10_num_obs"] = int(df_clean.shape[0])

Before cleaning: (74, 9)
After cleaning : (46, 9)



### Exercise 11

Now that we've established we have good balance (meaning we think randomization was likely successful), we can evaluate the effects of the experiment. Test whether the OEC and the metric you *don't* want affected have different average values in the control group and treatment group. 

Because we've randomized, this is a consistent estimate of the Average Treatment Effect of Udacity's website change.

Calculate the difference in means in your OEC and guardrail metrics using a simple t-test. Store the resulting effect estimates in `ex11_oec_ate` and `ex11_guard_ate` and p-values in `ex11_oec_pvalue` and `ex11_guard_pvalue`. **Please round all answers to 4 decimal places.** Report your ATE in *percentage points*, where `1` denotes 1 percentage point.


In [17]:
from scipy import stats
import numpy as np

oec_ctrl_vec = df_clean.loc[df_clean["treated"] == 0, "click_diff_ratio"].values
oec_treat_vec = df_clean.loc[df_clean["treated"] == 1, "click_diff_ratio"].values

oec_ate = (np.mean(oec_treat_vec) - np.mean(oec_ctrl_vec)) * 100
oec_pval = stats.ttest_ind(oec_treat_vec, oec_ctrl_vec, equal_var=False).pvalue

results["ex11_oec_ate"] = round(float(oec_ate), 4)
results["ex11_oec_pvalue"] = round(float(oec_pval), 4)

guard_ctrl_vec = df_clean.loc[df_clean["treated"] == 0, "click_pay_ratio"].values
guard_treat_vec = df_clean.loc[df_clean["treated"] == 1, "click_pay_ratio"].values

guard_ate = (np.mean(guard_treat_vec) - np.mean(guard_ctrl_vec)) * 100
guard_pval = stats.ttest_ind(guard_treat_vec, guard_ctrl_vec, equal_var=False).pvalue

results["ex11_guard_ate"] = round(float(guard_ate), 4)
results["ex11_guard_pvalue"] = round(float(guard_pval), 4)

print("OEC ATE (pp):", results["ex11_oec_ate"], "p-value:", results["ex11_oec_pvalue"])
print(
    "Guardrail ATE (pp):",
    results["ex11_guard_ate"],
    "p-value:",
    results["ex11_guard_pvalue"],
)

OEC ATE (pp): 1.5888 p-value: 0.132
Guardrail ATE (pp): -0.4897 p-value: 0.5928


### Exercise 12

Do you feel that Udacity achieved their goal? Did their intervention cause them any problems? If they asked you "What would happen if we rolled this out to everyone?" what would you say?

As you answer this question, a small additional question: up until this point you've (presumably) been reporting the default p-values from the tools you are using. These, as you may recall from stats 101, are two-tailed p-values. Do those seem appropriate for your OEC?

Based on the results, Udacity appears to have largely achieved the goal. The treatment changes the OEC in the intended direction, which suggest that setting clearer expectations upfront reduced the mismatch between enrolling in the free trial and ultimately paying. At the same time, the guardrail metric does not show a meaningful or statistically significant change, which suggests that the intervention did not create obvious negative side effects.

If Udacity were to roll this change out to everyone, we would expect a small improvement in the primary outcome without significantly harming monetization. One caveat is that the reported p-values are two-tailed, but Udacity’s hypothesis about the OEC is directional. As a result, a one-tailed test could be more appropriate for the OEC, although using two-tailed p-values is a more conservative choice.

### Exercise 13

One of the magic things about experiments is that all you have to do is compare averages to get an average treatment effect. However, you *can* do other things to try and increase the statistical power of your experiments, like add controls in a linear regression model. 

As you likely know, a bivariate regression is exactly equivalent to a t-test, so let's start by re-estimating the effect of treatment on your OEC using a linear regression. Can you replicate the results from your t-test? They shouldn't just be close—they should be numerically equivalent (i.e. exactly the same to the limits of floating point number precision). 

In [18]:
import statsmodels.api as sm

y = df_clean["click_diff_ratio"].values
X = sm.add_constant(df_clean["treated"].values)

model = sm.OLS(y, X).fit()
beta_hat = model.params[1]
beta_hat_pp = beta_hat * 100

print(model.summary())
print("\nRegression ATE (pp):", beta_hat_pp)
print("Ex11 t-test ATE (pp):", results["ex11_oec_ate"])

print("Matches (isclose):", np.isclose(beta_hat_pp, results["ex11_oec_ate"]))

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.051
Model:                            OLS   Adj. R-squared:                  0.029
Method:                 Least Squares   F-statistic:                     2.356
Date:                Mon, 02 Feb 2026   Prob (F-statistic):              0.132
Time:                        16:49:49   Log-Likelihood:                 89.832
No. Observations:                  46   AIC:                            -175.7
Df Residuals:                      44   BIC:                            -172.0
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.1021      0.007    -13.948      0.0

The regression reproduces the same treatment effect as the t-test: the coefficient on treated is about 0.0159 (≈ 1.5888 percentage points when expressed in pp), which matches the Ex11 ATE up to rounding. The isclose check is returning False only because the regression ATE is shown with full floating-point precision while the Ex11 ATE was rounded to 4 decimals—after rounding, they agree. The p-value for the treatment coefficient is 0.132, so the estimated improvement in the OEC is not statistically significant at conventional levels.

### Exercise 14

Now add indicator variables for the date of each observation. Do the standard errors on your `treatment` variable change? If so, in what direction?

Store your new standard error in `ex14_se_treatment`. Round your answer to 4 decimal places.

You should have found that your standard errors decreased by about 30\%—this is why, although just comparing means *works*, if you have additional variables adding them to your analysis can be helpful (all the usual rules for model specification apply — for example, you still want to be careful about overfitting, which one could argue is maybe part of what's happening here). 

In many other cases, the effect of adding controls is likely to be larger — the date indicators we added to our data are perfectly balanced between treatment and control, so we aren't adding a lot of data to the model by adding them as variables. They're accounting for some day-to-day variation (presumably in the types of people coming to the site), but they aren't controlling for any residual baseline differences the way a control like "gender" or "age" might (since those kind of individual-level attributes will never be perfectly balanced across treatment and control). 

In [20]:
import statsmodels.formula.api as smf

df_clean["Date"] = df_clean["Date"].astype("category")
fe_model = smf.ols("click_diff_ratio ~ treated + C(Date)", data=df_clean).fit()
se_treated = fe_model.bse["treated"]
print("SE(treated) with date indicators:", se_treated)
results["ex14_se_treatment"] = round(float(se_treated), 4)

SE(treated) with date indicators: 0.00662537652255688


After adding date indicator variables, the standard error on the treatment coefficient drops to about 0.0066. Compared with the bivariate model, this is smaller, meaning the estimate becomes more precise because the date dummies absorb some day-to-day fluctuations in outcomes that would otherwise show up as noise in the error term.

### Exercise 15

Does this result have any impact on the recommendations you would offer Udacity?

Yes, because adding date indicators makes the treatment effect estimate more precise, which increases our confidence in the direction and size of the estimated effect. However, it doesn’t fundamentally change the recommendation: we still shouldn’t treat the result as a guaranteed win unless the estimated effect is clearly meaningful and statistically convincing. What it does suggest is that Udacity should rely on the regression-with-controls to make a more confident rollout decision.